# 환경설정

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

### 1. 문서 로드 (Load Documents)

In [3]:
!pip install pymupdf

   ---------------------------------------- 0.0/16.0 MB ? eta -:--:--
   -------------------- ------------------- 8.4/16.0 MB 43.5 MB/s eta 0:00:01
   ------------------------------------ --- 14.7/16.0 MB 51.3 MB/s eta 0:00:01
   ---------------------------------------  15.7/16.0 MB 33.0 MB/s eta 0:00:01
   ---------------------------------------- 16.0/16.0 MB 20.1 MB/s eta 0:00:00


In [4]:
from langchain_community.document_loaders import PyMuPDFLoader

# PYMUPDFLoader 객체 정의
loader = PyMuPDFLoader("data/snow-white.pdf")

# 문서 로드
docs = loader.load()

print(f"문서의 페이지 수 : {len(docs)}")

문서의 페이지 수 : 6


In [5]:
print(docs[0].page_content)

백설공주
옛날어느왕국에공주님이태어났어요.
“어쩜이렇게어여쁠까? 살결이눈처럼하얗구나. 백
설공주라고불러야겠다.”
왕과왕비는갓태어난딸을보며기뻐했어요.
하지만기쁨도잠시, 왕비는곧세상을떠나고말았어
요.



In [6]:
# 메타 데이터
print(docs[0].__dict__)

{'id': None, 'metadata': {'source': 'data/snow-white.pdf', 'file_path': 'data/snow-white.pdf', 'page': 0, 'total_pages': 6, 'format': 'PDF 1.5', 'title': 'PowerPoint 프레젠테이션', 'author': 'PC', 'subject': '', 'keywords': '', 'creator': 'Microsoft® PowerPoint® 2013', 'producer': 'Microsoft® PowerPoint® 2013', 'creationDate': "D:20230912112024+09'00'", 'modDate': "D:20230912112024+09'00'", 'trapped': ''}, 'page_content': '백설공주\n옛날어느왕국에공주님이태어났어요.\n“어쩜이렇게어여쁠까? 살결이눈처럼하얗구나. 백\n설공주라고불러야겠다.”\n왕과왕비는갓태어난딸을보며기뻐했어요.\n하지만기쁨도잠시, 왕비는곧세상을떠나고말았어\n요.\n', 'type': 'Document'}


### 2. 문서 분할(split Documents)

In [8]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=10)

split_documents = text_splitter.split_documents(docs)

print(f"분할된 청크의 수 : {len(split_documents)}")

분할된 청크의 수 : 21


### 3. 임베딩 (Embedding 생성)

In [9]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

4. DB 생성 (벡터스토어 생성) 및 저장
- FAISS(Facebook AI Similarity Search)
    - 페이스북에서 개발한 유사도 검색 및 클러스터링 라이브러리
    - 벡터 데이터셋에서 빠른 유사도 검색

In [11]:
!pip install faiss-cpu

   ---------------------------------------- 0.0/14.9 MB ? eta -:--:--
   ------------------------ --------------- 9.2/14.9 MB 47.4 MB/s eta 0:00:01
   ---------------------------------------- 14.9/14.9 MB 36.0 MB/s eta 0:00:00


In [12]:
from langchain_community.vectorstores import FAISS

vectorstore = FAISS.from_documents(documents=split_documents, embedding=embeddings)

In [13]:
for doc in vectorstore.similarity_search("난쟁이"):
    print(doc.page_content)

저녁이되자, 일곱난쟁이가돌아왔어요.
난쟁이들은쓰러진백설공주를보고엉엉울었어요.
백설공주는깊은잠에빠진것처럼보였지요.
“백설공주님, 못된왕비의꾐에넘어갔군요.”
왕자는깨어난백설공주를보고기뻐했어요.
“공주님, 나는이웃나라왕자입니다.”
“왕자님이나를다시살려주셨군요.”
“나와결혼해주시겠어요?”
“네, 좋아요!”
간사과랍니다. 잠깐문을열어보세요.”
백설공주는고개를저었어요.
“난쟁이들이문을열어주지말라고했어요.”
백설공주가거절하자, 왕비는창문틈새로사과를쑥내밀었어
요.
밤이되자오두막주인인일곱난쟁이가돌아왔어요.
난쟁이들은집안이어질러진것을보고깜짝놀랐지요.
일곱째난쟁이가큰소리로외쳤어요.
“누가내침대에서자고있어!”


5. 검색기(Retriever) 생성

In [14]:
# 벡터스토어에 있는 정보를 검색하고 생성
retriever = vectorstore.as_retriever()

In [15]:
retriever.invoke("백설공주와 일곱난쟁이는 어디서 만났어?")

[Document(metadata={'source': 'data/snow-white.pdf', 'file_path': 'data/snow-white.pdf', 'page': 0, 'total_pages': 6, 'format': 'PDF 1.5', 'title': 'PowerPoint 프레젠테이션', 'author': 'PC', 'subject': '', 'keywords': '', 'creator': 'Microsoft® PowerPoint® 2013', 'producer': 'Microsoft® PowerPoint® 2013', 'creationDate': "D:20230912112024+09'00'", 'modDate': "D:20230912112024+09'00'", 'trapped': ''}, page_content='백설공주\n옛날어느왕국에공주님이태어났어요.\n“어쩜이렇게어여쁠까? 살결이눈처럼하얗구나. 백\n설공주라고불러야겠다.”\n왕과왕비는갓태어난딸을보며기뻐했어요.'),
 Document(metadata={'source': 'data/snow-white.pdf', 'file_path': 'data/snow-white.pdf', 'page': 4, 'total_pages': 6, 'format': 'PDF 1.5', 'title': 'PowerPoint 프레젠테이션', 'author': 'PC', 'subject': '', 'keywords': '', 'creator': 'Microsoft® PowerPoint® 2013', 'producer': 'Microsoft® PowerPoint® 2013', 'creationDate': "D:20230912112024+09'00'", 'modDate': "D:20230912112024+09'00'", 'trapped': ''}, page_content='저녁이되자, 일곱난쟁이가돌아왔어요.\n난쟁이들은쓰러진백설공주를보고엉엉울었어요.\n백설공주는깊은잠에빠진것처럼보였지요.\n“백설공주님, 못된왕비의꾐에넘어갔군요.

### 6. 프롬프트 생성

당신은 질문 - 답변 작업을 위한 어시스턴트입니다.  
주어진 문맥을 사용하여 질문에 답변하세요.  
유치원 선생님이 아이에게 말하는 것처럼 매우 친절하고 부드러운 어조를 사용하세요.  
따듯하고 친근한 방식으로 말하세요.  
답을 모르는 경우에는 모른다고 말씀하세요.  
한국어로 답변하세요.

In [16]:
from langchain_core.prompts import PromptTemplate
prompt = PromptTemplate.from_template(
    """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question.
Use a very kind and gentle tone like a kindergarten teacher talking to a child.
Speak in a warm and friendly way.
If you don't know the answer, just say that you don't know. 
Answer in Korean.

#Context: 
{context}

#Question:
{question}

#Answer:"""
)

### LLM 모델 생성

In [19]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o", temperature=0)

### 8. Chain 생성

In [22]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
# RunnablePassthrough 그대로 전달해주는 것 

chain = (
    {"context" : retriever, "question" : RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [26]:
# 체인 실행

question = "제일 큰 난쟁이는 누구야?"
response = chain.invoke(question)

print(response)

미안하지만, 제일 큰 난쟁이가 누구인지에 대한 정보는 여기 없어요. 하지만 괜찮아요, 다른 재미있는 이야기를 찾아볼 수 있답니다!


---

## 실습

In [41]:
# 문서 로드
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import PromptTemplate

# PYMUPDFLoader 객체 정의
loader = PyMuPDFLoader("data/SPRi AI Brief_10.pdf")

# 문서 로드
docs = loader.load()

print(f"문서의 페이지 수 : {len(docs)}")

# 문서 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=10)

split_documents = text_splitter.split_documents(docs)

print(f"분할된 청크의 수 : {len(split_documents)}")

# 임베딩
embeddings = OpenAIEmbeddings()

vectorstore = FAISS.from_documents(documents=split_documents, embedding=embeddings)
    
# 벡터스토어에 있는 정보를 검색하고 생성
retriever = vectorstore.as_retriever()

prompt = PromptTemplate.from_template(
    """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know. 
Answer in Korean.

#Context: 
{context}

#Question:
{question}

#Answer:"""
)

from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o", temperature=0)

from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
# RunnablePassthrough 그대로 전달해주는 것 

chain = (
    {"context" : retriever, "question" : RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# 체인 실행

question = "대략적으로 전체적인 내용을 요약해줘"
response = chain.invoke(question)

print(response)

문서의 페이지 수 : 25
분할된 청크의 수 : 499
보고서는 AI 국제과학 패널 구성, AI 정책 대화, AI 표준 교류, AI 역량 개발 네트워크 형성 등 글로벌 AI 거버넌스의 격차 해소를 위한 7가지 권고안을 제시하고 있습니다. 또한, AI의 추론 역량이 물리학, 화학, 생물학의 복잡한 과제에서 박사과정 학생과 비슷한 성과를 나타내며, 수학과 코딩에서도 탁월한 성과를 기록하고 있습니다. 다만, 다국어 작업에는 최적화되지 않아 추가적인 미세조정이 필요하다는 점도 언급하고 있습니다.


In [9]:
import bs4
from dotenv import load_dotenv
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough 

# 환경 변수 가져오기
load_dotenv()

# 문서 로드
loader = WebBaseLoader(
    web_path=("https://n.news.naver.com/article/437/0000416134"),
    bs_kwargs=dict(
        # 특정 요소에서만 파싱하도록 
        parse_only=bs4.SoupStrainer(
            "div",
            attrs={"class" : ["newsct_article _article_body", "media_end_head go_trans"]}
        )
    )
)

docs = loader.load()

# print(docs)

# 문서 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=10)

splits = text_splitter.split_documents(docs)

# 임베딩 생성

embeddings = OpenAIEmbeddings()

# 벡터스토어 생성
vectorstore = FAISS.from_documents(documents=splits, embedding=embeddings)

# 검색기 (retriever) 생성
retriever = vectorstore.as_retriever()

# runtime

# 프롬프트
prompt = PromptTemplate.from_template(
    """
    당신은 질문-답변을 수행하는 AI 어시스턴트이다.
    주어진 문맥에 검색된 다음 문맥(context)를 사용해 질문에 답해야 한다.
    만약 주어진 문맥에서 답을 찾을 수 없는 경우, 모른다고 이야기 하세요.
    한글로 답변해주세요.
    
    #Question:
    {question}
    
    #Context:
    {context}
    
    #Answer:
    """
)

# LLM 모델
llm = ChatOpenAI(model="gpt-4o", temperature=0)

# Chain 구성
news_chain = (
    {"context":retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# 질문
answer = news_chain.invoke("아파트가 왜 인기 있는 거야?")

print(answer)

모르겠습니다. 주어진 문맥에서는 아파트가 인기 있는 이유에 대한 정보가 포함되어 있지 않습니다.
